In [0]:
# 💡 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐨𝐭𝐚𝐥 𝐭𝐢𝐦𝐞 𝐬𝐩𝐞𝐧𝐭 𝐩𝐞𝐫 𝐬𝐞𝐬𝐬𝐢𝐨𝐧?
# 💡 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐨𝐭𝐚𝐥 𝐬𝐞𝐬𝐬𝐢𝐨𝐧 𝐭𝐢𝐦𝐞 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫?

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,lag,count,sum, to_timestamp

spark = SparkSession.builder.master("local[1]").appName("SessionTime").getOrCreate()

#sample  customer interaction data
data = [
 ("user1", "2023-08-26 10:10:00"),
 ("user1", "2023-08-26 10:10:25"),
 ("user2", "2023-08-26 12:00:00"),
 ("user2", "2023-08-26 12:10:00"),
 ("user1", "2023-08-26 14:30:00"),
 ("user1", "2023-08-26 16:00:00"),
 ("user2", "2023-08-26 16:30:00"),
 ("user1", "2023-08-26 18:00:00"),
]
schema = StructType([
 StructField("userid", StringType(), nullable=False), StructField("timestamp", StringType(), nullable=False)])

#create  Dataframe

sessions_df = spark.createDataFrame(data, schema)
sessions_df = sessions_df.withColumn('timestamp', to_timestamp(col('timestamp')))
sessions_df.show(truncate=False)
w = Window().partitionBy('userid').orderBy('timestamp')
sessions_df = sessions_df.withColumn('lag_session',lag('timestamp').over(w))
sessions_df = sessions_df.withColumn('total_time_per_session', col('timestamp').cast('long') - col('lag_session').cast('long'))
sessions_df = sessions_df.where(col('lag_session').isNotNull())
sessions_df.show()
sessions_df = sessions_df.groupBy('userid').agg(sum('total_time_per_session').alias('total_sessiontime_per_user')).show()

+------+-------------------+
|userid|timestamp          |
+------+-------------------+
|user1 |2023-08-26 10:10:00|
|user1 |2023-08-26 10:10:25|
|user2 |2023-08-26 12:00:00|
|user2 |2023-08-26 12:10:00|
|user1 |2023-08-26 14:30:00|
|user1 |2023-08-26 16:00:00|
|user2 |2023-08-26 16:30:00|
|user1 |2023-08-26 18:00:00|
+------+-------------------+

+------+-------------------+-------------------+----------------------+
|userid|          timestamp|        lag_session|total_time_per_session|
+------+-------------------+-------------------+----------------------+
| user1|2023-08-26 10:10:25|2023-08-26 10:10:00|                    25|
| user1|2023-08-26 14:30:00|2023-08-26 10:10:25|                 15575|
| user1|2023-08-26 16:00:00|2023-08-26 14:30:00|                  5400|
| user1|2023-08-26 18:00:00|2023-08-26 16:00:00|                  7200|
| user2|2023-08-26 12:10:00|2023-08-26 12:00:00|                   600|
| user2|2023-08-26 16:30:00|2023-08-26 12:10:00|                 15600|
+--

In [0]:
# Write a pyspark dataframe query to find all customers who never order anything.

# Table: Customers.

# +----+-------+ | Id | NameCust | +----+-------+ | 1 | Joe | | 2 | Henry | | 3 | Sam | | 4 | Max | +----+-------+

# Table: Orders.

# +----+------------+ | Id | CustomerId | +----+------------+ | 1 | 3 | | 2 | 1 | +----+------------+ Using the above tables as example, return the following:

# +-----------+ | Customers | +-----------+ | Henry | | Max | +-----------+

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,lag,count,sum

spark = SparkSession.builder.master("local[1]").appName("Customer").getOrCreate()
spark1 = SparkSession.builder.master("local[1]").appName("Order").getOrCreate()

#sample  customer interaction data
data = [
 (1, "Joe"),
 (2, "Henry"),
 (3, "Sam"),
 (4, "Max")
]
schema = StructType([
 StructField("Id", IntegerType(), nullable=False), StructField("NameCust", StringType(), nullable=False)])


data1 = [
 (1, 3),
 (2, 1)
]
schema1 = StructType([
 StructField("Id", IntegerType(), nullable=False), StructField("CustomerId", IntegerType(), nullable=False)])

#create  Dataframe

customer_df = spark.createDataFrame(data, schema)
customer_df.show(truncate=False)

order_df = spark.createDataFrame(data1, schema1)
order_df.show(truncate=False)

#joining the datasets
joined_df = customer_df.join(order_df,customer_df.Id == order_df.CustomerId, 'left')
joined_df.show()

#filter
filterdf = joined_df.filter(joined_df.CustomerId.isNull())
filterdf.show()

#result
result_df = filterdf.select('NameCust')
result_df.show()

+---+--------+
|Id |NameCust|
+---+--------+
|1  |Joe     |
|2  |Henry   |
|3  |Sam     |
|4  |Max     |
+---+--------+

+---+----------+
|Id |CustomerId|
+---+----------+
|1  |3         |
|2  |1         |
+---+----------+

+---+--------+----+----------+
| Id|NameCust|  Id|CustomerId|
+---+--------+----+----------+
|  1|     Joe|   2|         1|
|  2|   Henry|null|      null|
|  3|     Sam|   1|         3|
|  4|     Max|null|      null|
+---+--------+----+----------+

+---+--------+----+----------+
| Id|NameCust|  Id|CustomerId|
+---+--------+----+----------+
|  2|   Henry|null|      null|
|  4|     Max|null|      null|
+---+--------+----+----------+

+--------+
|NameCust|
+--------+
|   Henry|
|     Max|
+--------+



In [0]:
# Write a pyspark query that reports for every date within at most 90 days from login_date, the number of users that logged in for the first time on that date. Login_date is 2019-06-30.

# Traffic table: +---------+----------+---------------+ | user_id | activity | activity_date | +---------+----------+---------------+ | 1 | login | 2019-05-01 | | 1 | homepage | 2019-05-01 | | 1 | logout | 2019-05-01 | | 2 | login | 2019-06-21 | | 2 | logout | 2019-06-21 | | 3 | login | 2019-01-01 | | 3 | jobs | 2019-01-01 | | 3 | logout | 2019-01-01 | | 4 | login | 2019-06-21 | | 4 | groups | 2019-06-21 | | 4 | logout | 2019-06-21 | | 5 | login | 2019-03-01 | | 5 | logout | 2019-03-01 | | 5 | login | 2019-06-21 | | 5 | logout | 2019-06-21 | +---------+----------+---------------+

# Result table: +------------+-------------+ | login_date | user_count | +------------+-------------+ | 2019-05-01 | 1 | | 2019-06-21 | 2 | +------------+-------------+


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,lag,count,sum, date_sub, lit

spark = SparkSession.builder.master("local[1]").appName("Reports").getOrCreate()

#sample  customer interaction data
data = [
 (1,'login','2019-05-01'),
 (1,'homepage','2019-05-01'),
 (1,'logout','2019-05-01'),
 (2,'login','2019-06-21'),
 (2,'logout','2019-06-21'),
 (3,'login','2019-01-01'),
 (3,'jobs','2019-01-01'),
 (3,'logout','2019-01-01'),
 (4,'login','2019-06-21'),
 (4,'groups','2019-06-21'),
 (4,'logout','2019-06-21'),
 (5,'login','2019-03-01'),
 (5,'logout','2019-03-01'),
 (5,'login','2019-06-21'),
 (5,'logout','2019-06-21'),
]
schema = StructType([
 StructField("user_id", IntegerType(), nullable=False), StructField("activity", StringType(), nullable=False),
 StructField("activity_date", StringType(), nullable=False)])



#create  Dataframe

report_df = spark.createDataFrame(data, schema)
report_df.show(truncate=False)


# Calculate the date range within at most 90 days from the provided date (2019-06-30)
#lit - add literal/constant to existing dataframe
#date_sub - difference in dates 
max_diff  = date_sub(lit('2019-06-30'),90)

date_range_df = report_df.filter((col('activity_date') >= max_diff) & (col('activity_date') <= '2019-06-30'))
date_range_df.show()


# Find the first login date for each user
####way to change column name after aggregrate function
first_login_df = date_range_df.filter(col('activity') == 'login').groupBy('user_id').agg(min('activity_date')).withColumnRenamed('min(activity_date)', 'login_date_first')
first_login_df.show()

# Count the number of users for each login date
count_df = first_login_df.groupBy('login_date_first').count()

# Show the result
count_df.show()

+-------+--------+-------------+
|user_id|activity|activity_date|
+-------+--------+-------------+
|1      |login   |2019-05-01   |
|1      |homepage|2019-05-01   |
|1      |logout  |2019-05-01   |
|2      |login   |2019-06-21   |
|2      |logout  |2019-06-21   |
|3      |login   |2019-01-01   |
|3      |jobs    |2019-01-01   |
|3      |logout  |2019-01-01   |
|4      |login   |2019-06-21   |
|4      |groups  |2019-06-21   |
|4      |logout  |2019-06-21   |
|5      |login   |2019-03-01   |
|5      |logout  |2019-03-01   |
|5      |login   |2019-06-21   |
|5      |logout  |2019-06-21   |
+-------+--------+-------------+

+-------+--------+-------------+
|user_id|activity|activity_date|
+-------+--------+-------------+
|      1|   login|   2019-05-01|
|      1|homepage|   2019-05-01|
|      1|  logout|   2019-05-01|
|      2|   login|   2019-06-21|
|      2|  logout|   2019-06-21|
|      4|   login|   2019-06-21|
|      4|  groups|   2019-06-21|
|      4|  logout|   2019-06-21|
|      5|

In [0]:
# Write a Pyspark query to report the movies with an odd-numbered ID and a description that is not "boring".Return the result table in descending order by rating.

# Cinema table: +----+------------+-------------+--------+ | id | movie | description | rating | +----+------------+-------------+--------+ | 1 | War | great 3D | 8.9 | | 2 | Science | fiction | 8.5 | | 3 | irish | boring | 6.2 | | 4 | Ice song | Fantacy | 8.6 | | 5 | House card | Interesting | 9.1 | +----+------------+-------------+--------+

# Result table: +----+------------+-------------+--------+ | id | movie | description | rating | +----+------------+-------------+--------+ | 5 | House card | Interesting | 9.1 | | 1 | War | great 3D | 8.9 | +----+------------+-------------+--------+


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,lag,count,sum

spark = SparkSession.builder.master("local[1]").appName("Cinema").getOrCreate()

#sample  customer interaction data
data = [
 (1,'War','Great 3D',8.9),
 (2,'Science','Fiction',8.5),
 (3,'Irish','Boring',6.2),
 (4,'Ice song','Fantacy',8.6),
 (5,'House card','Interesting',9.1)
]
schema = StructType([
 StructField("id", IntegerType(), nullable=False), StructField("movie", StringType(), nullable=False),
 StructField("description", StringType(), nullable=False),StructField("rating", FloatType(), nullable=False)])


#create  Dataframe

cinema_df = spark.createDataFrame(data, schema)
cinema_df.show(truncate=False)

result_df = cinema_df.filter( (cinema_df.id%2 != 0 ) & (cinema_df.description != 'Boring'))
result_df.show()


In [0]:
#11
# Question: Calculate Average User Session Duration
# You have a dataset containing user activity logs in a PySpark DataFrame with the following columns: user_id, timestamp, and action. The action column indicates whether the user started or ended a session. It can have values 'start' or 'end'. Your task is to calculate the average duration of user sessions.

# user_id | timestamp          | action
# ------- | ------------------ | ------
# 1       | 2022-01-01 10:00   | start
# 1       | 2022-01-01 10:15   | end
# 2       | 2022-01-01 11:00   | start
# 1       | 2022-01-01 11:30   | start
# 2       | 2022-01-01 11:45   | end
# 1       | 2022-01-01 12:00   | end


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,lag,count,sum, to_timestamp, when,cast, to_utc_timestamp

spark = SparkSession.builder.master("local[1]").appName("User_session").getOrCreate()

#sample  customer interaction data
data = [
        (1," start"," 2022-01-01 10:00"),
        (1," end"," 2022-01-01 10:15"),
        (2," start"," 2022-01-01 11:00"),
        (1," start"," 2022-01-01 11:30"),
        (2," end", " 2022-01-01 11:45"),
        (1," end"," 2022-01-01 12:00")
]
schema = StructType([
 StructField("user_id", IntegerType(), nullable=False), StructField("action", StringType(), nullable=False),
 StructField("timestamp", StringType(), nullable=False)])

#create  Dataframe

user_df = spark.createDataFrame(data, schema)
user_df.show(truncate=False)

#Logic

user_df = user_df.withColumn('unix_timestamp',unix_timestamp(col('timestamp'),'yyyy-MM-dd HH:mm:ss'))
user_df.show()


#Calculate session duration
w  = Window().partitionBy('user_id').orderBy('unix_timestamp')
user_df = user_df.withColumn('session_duration', when(user_df.action =='end', col('unix_timestamp').cast('long') - lag(col('unix_timestamp').over(w).cast('long'))))
user_df.show()

#Calculate total session duration per user
user_session_duration = user_df.groupBy('user_id').agg(sum('session_duration').alias('total_duration_per_user'))
user_session_duration.show()

#Calculate Averagre Session duration
Avg_duration = user_session_duration.selectExpr('user_id','total_duration_per_user / count(user_id) as Avg_duration')
Avg_duration.show()


+-------+------+-----------------+
|user_id|action|timestamp        |
+-------+------+-----------------+
|1      | start| 2022-01-01 10:00|
|1      | end  | 2022-01-01 10:15|
|2      | start| 2022-01-01 11:00|
|1      | start| 2022-01-01 11:30|
|2      | end  | 2022-01-01 11:45|
|1      | end  | 2022-01-01 12:00|
+-------+------+-----------------+

+-------+------+-----------------+--------------+
|user_id|action|        timestamp|unix_timestamp|
+-------+------+-----------------+--------------+
|      1| start| 2022-01-01 10:00|          null|
|      1|   end| 2022-01-01 10:15|          null|
|      2| start| 2022-01-01 11:00|          null|
|      1| start| 2022-01-01 11:30|          null|
|      2|   end| 2022-01-01 11:45|          null|
|      1|   end| 2022-01-01 12:00|          null|
+-------+------+-----------------+--------------+



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2484405975002356>:48
     46 #Calculate session duration
     47 w  = Window().partitionBy('user_id').orderBy('unix_timestamp')
---> 48 user_df = user_df.withColumn('session_duration', when(user_df.action =='end', col('unix_timestamp').cast('long') - lag(col('unix_timestamp').over(w).cast('long'))))
     49 user_df.show()
     51 #Calculate total session duration per user

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:4758, in DataFrame.withColumn(

In [0]:
# 11
# Sample Question: Analyzing Sales Data
# Problem Statement: You are given a CSV file containing sales data with the following columns: date, product_id, quantity_sold, and price_per_unit. Your task is to write a PySpark program to analyze this sales data and produce the following insights:
# •	Total revenue for each product.
# •	Total revenue for each date.
# •	Top 5 selling products based on the quantity sold.

# date,product_id,quantity_sold,price_per_unit
# 2023-08-01,101,5,10.0
# 2023-08-01,102,3,15.0
# 2023-08-02,101,8,10.0
# 2023-08-02,103,6,12.0
# 2023-08-03,102,4,15.0
# 2023-08-03,104,2,20.0

# Expected output

# Total revenue per product:
# +----------+-------------+
# |product_id|total_revenue|
# +----------+-------------+
# |       101|         90.0|
# |       102|         75.0|
# |       103|         72.0|
# |       104|         40.0|
# +----------+-------------+

# Total revenue per date:
# +----------+-------------+
# |      date|total_revenue|
# +----------+-------------+
# |2023-08-01|         135.0|
# |2023-08-02|         150.0|
# |2023-08-03|         110.0|
# +----------+-------------+

# Top 5 selling products:
# +----------+-------------+
# |product_id|total_quantity_sold|
# +----------+-------------+
# |       101|                13|
# |       102|                 7|
# |       103|                 6|
# |       104|                 2|
# +----------+-------------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,lag,count,sum

spark = SparkSession.builder.master("local[1]").appName("User_session").getOrCreate()

#sample  customer interaction data
data = [
        ('2023-08-01',101,5,10.0),
        ('2023-08-01',102,3,15.0),
        ('2023-08-02',101,8,10.0),
        ('2023-08-02',103,6,12.0),
        ('2023-08-03',102,4,15.0),
        ('2023-08-03',104,2,20.0)
]
schema = StructType([
 StructField("date", StringType(), nullable=False), StructField("product_id", StringType(), nullable=False),
 StructField("quantity_sold", StringType(), nullable=False), StructField("price_per_unit", StringType(), nullable=False)])

#create  Dataframe

user_df = spark.createDataFrame(data, schema)
user_df.show(truncate=False)

#Logic
# •	Total revenue for each product.
user_df.withColumn('total_revenue',col('quantity_sold') * col('price_per_unit')).groupBy('product_id').agg(sum('total_revenue').alias('total_revenue')).show()

# Total revenue per date:
user_df.withColumn('total_revenue_per_date',col('quantity_sold') * col('price_per_unit')).groupBy('date').agg(sum('total_revenue_per_date').alias('total_revenue')).show()

# Top 5 selling products:
user_df.groupBy('product_id').agg(sum('quantity_sold').alias('total_quantity_sold')).limit(5).show()


+----------+----------+-------------+--------------+
|date      |product_id|quantity_sold|price_per_unit|
+----------+----------+-------------+--------------+
|2023-08-01|101       |5            |10.0          |
|2023-08-01|102       |3            |15.0          |
|2023-08-02|101       |8            |10.0          |
|2023-08-02|103       |6            |12.0          |
|2023-08-03|102       |4            |15.0          |
|2023-08-03|104       |2            |20.0          |
+----------+----------+-------------+--------------+

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       101|        130.0|
|       102|        105.0|
|       103|         72.0|
|       104|         40.0|
+----------+-------------+

+----------+-------------+
|      date|total_revenue|
+----------+-------------+
|2023-08-01|         95.0|
|2023-08-02|        152.0|
|2023-08-03|        100.0|
+----------+-------------+

+----------+-------------------+
|product_id|total_quantity_so

In [0]:
#12

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("TopConsecutiveUsers").getOrCreate()

# Define the schema for your data
schema = ["user_id", "timestamp"]
data = [
    ("user1", "2023-08-26 10:10:00"),
    ("user1", "2023-08-26 10:10:25"),
    ("user2", "2023-08-26 12:00:00"),
    ("user2", "2023-08-26 12:10:00"),
    ("user1", "2023-08-27 14:30:00"),
    ("user1", "2023-08-28 16:00:00"),
    ("user2", "2023-08-27 16:30:00"),
    ("user1", "2023-08-29 18:00:00"),
]

# Create a DataFrame from the data
df = spark.createDataFrame(data, schema=schema)
df.show()

# Convert the timestamp column to a date type
df = df.withColumn('date',F.to_date('timestamp'))
df.show()
# Define a window specification to partition by user and order by date
w = Window().partitionBy('user_id').orderBy('date')

# Calculate the difference between consecutive dates
df = df.withColumn('date_diff', F.datediff('date',F.lag('date').over(w)))
df.show()
# Identify consecutive login sequences
df = df.withColumn('is_consecutive',F.when(F.col('date_diff') == 1, 1).otherwise(0))

# Calculate the cumulative sum of consecutive logins
df= df.withColumn('cumulative_sum',F.sum('is_consecutive').over(w))

# Find the maximum consecutive logins for each user
max_consecutive_logins = df.groupBy('user_id').agg(max('cumulative_sum').alias('max_consecutive_days'))
max_consecutive_logins.show()

# Find the top 5 users with the most consecutive logins
top_5 = max_consecutive_logins.orderBy(F.desc('max_consecutive_days')).limit(5)

# Show the result
top_5.show()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-26 10:10:00|
|  user1|2023-08-26 10:10:25|
|  user2|2023-08-26 12:00:00|
|  user2|2023-08-26 12:10:00|
|  user1|2023-08-27 14:30:00|
|  user1|2023-08-28 16:00:00|
|  user2|2023-08-27 16:30:00|
|  user1|2023-08-29 18:00:00|
+-------+-------------------+

+-------+-------------------+----------+
|user_id|          timestamp|      date|
+-------+-------------------+----------+
|  user1|2023-08-26 10:10:00|2023-08-26|
|  user1|2023-08-26 10:10:25|2023-08-26|
|  user2|2023-08-26 12:00:00|2023-08-26|
|  user2|2023-08-26 12:10:00|2023-08-26|
|  user1|2023-08-27 14:30:00|2023-08-27|
|  user1|2023-08-28 16:00:00|2023-08-28|
|  user2|2023-08-27 16:30:00|2023-08-27|
|  user1|2023-08-29 18:00:00|2023-08-29|
+-------+-------------------+----------+

+-------+-------------------+----------+---------+
|user_id|          timestamp|      date|date_diff|
+-------+-------------------+----------+----

In [0]:
# 13
# rolling averages

#  Qu: real-time scenario where you have a dataset of stock price data, and you want to calculate the rolling average stock price for each stock within a specific time window.

#  A rolling average, also known as a moving average, is a statistical calculation used in time series analysis to smooth out fluctuations in data over a specified period. It is a commonly used technique in various fields, including finance, economics, signal processing, and data analysis. The rolling average provides a more stable and less noisy representation of data, making it easier to identify trends and patterns.

# Here's how a rolling average is calculated:

# 1. Select a Window Size: You choose a fixed window size (e.g., 3 days, 7 days, 30 days) that defines the number of data points to include in each average calculation.

# 2. Slide the Window: Starting from the first data point, you slide the window over the time series data one step at a time. At each step, you include the data points within the window and calculate their average.

# 3. Smoothed Data: The result of this calculation is a new time series, often referred to as the rolling average or moving average. Each point in the rolling average represents the average value of the data within the window at that point in time.

# The rolling average is useful for various purposes:

# 1. Noise Reduction: It helps reduce random fluctuations or noise in time series data, making underlying trends more apparent.

# 2. Trend Identification: By smoothing the data, rolling averages make it easier to identify long-term trends, upward or downward movements, or cyclic patterns.

# 3. Seasonal Adjustment: In economics and finance, rolling averages are often used to seasonally adjust data by removing the seasonal components.

# 4. Forecasting: Rolling averages can be used as a basis for forecasting future values in time series analysis.

# Example:
# Let's say you have daily stock price data for a company, and you calculate a 7-day rolling average. The rolling average for a particular day would be the average of the stock prices for that day and the previous six days. This smooths out daily fluctuations, making it easier to see the overall trend in the stock's price.

# In PySpark, as shown in a previous response, you can use window functions to calculate rolling averages within a specified window size for time-based data analysis.